# 데이터 로딩

- glob, read_table 테스트


In [11]:
import glob, os
import pandas as pd

path = 'topics'
all_files = glob.glob(os.path.join(path, '*.data'))

# print(type(all_files))
# print(all_files)

df = pd.read_table(all_files[0], index_col=None, header =0,encoding='latin1')
print(df.shape)
display(df.head())
print(df.columns)

(66, 1)


,", and is very, very accurate ."
0,"but for the most part, we find that the Garmi..."
1,This function is not accurate if you don't le...
2,It provides immediate alternatives if the rou...
3,"I've used other GPS units, as well as GPS bui..."
4,It got me from point A to point B with 100% a...


Index([', and is very, very accurate .'], dtype='object')


In [13]:
pd.set_option('display.max_colwidth',700)

In [4]:
import glob, os
import pandas as pd

path = 'topics'
all_files = glob.glob(os.path.join(path, '*.data'))
filename_list = []
opinion_text = []

for file in all_files:
    df = pd.read_table(file, header = 0, encoding='latin1')
    filename = file.split('\\')[-1]
    filename = filename.split('.')[0]

    filename_list.append(filename)
    opinion_text.append(df.to_string())

document_df = pd.DataFrame({'filename':filename_list,'opinion':opinion_text})
document_df.head()


,filename,opinion
0,accuracy_garmin_nuvi_255W_gps,...
1,bathroom_bestwestern_hotel_sfo,...
2,battery-life_amazon_kindle,...
3,battery-life_ipod_nano_8gb,...
4,battery-life_netbook_1005ha,...


# TfidfVectorizer의 tokenizer 인자로 사용될 lemmatization 어근 변환 함수 선언

- translate() 함수 
    - 문자열 내에 특정 문자를 다른 문자로 일괄 변경하는 함수
    - 매개변수로 변경하고자 하는 정보를 딕셔너리로 전달한다.

In [1]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [19]:
# maketrans : 원래 문자와 바꿀 문자에 대한 맵핑정보(딕셔너리)를 만든다.
# 원본 문자열과 바꿀 문자열의 길이가 같아야 한다.
# {원본 문자 유니코드: 변경 문자 유니코드}

trans = str.maketrans('[],','   ')
print(trans)
text = '[고양이,강아지,다람쥐,토끼]'
text = text.translate(trans)

print(text)

{91: 32, 93: 32, 44: 32}
 고양이 강아지 다람쥐 토끼 


In [2]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punc_dict = {ord(punc):None for punc in string.punctuation}
lemmar = WordNetLemmatizer()

def LemNormalize(text):
    tokens = nltk.word_tokenize(text.lower().translate(remove_punc_dict))
    return [ lemmar.lemmatize(token) for token in tokens ]

# TF-IDF 기반 Vectorization 적용 및 KMeans 군집화 수행
- Stemming과 Lemmatization 같은 어근 변환은 TfidfVectorizer에서 직접 지원 하진 않으나 tokenizer 파라미터에 사용자 정의 어근 변환 함수를
- 적용하여 어근 변환을 수행할 수 있음
- TfidfVectorizer 생성자 파라미터
    - tokenizer : 토큰화를 별도의 사용자 정의 함수로 지정
    - max_df : 전체 문서에 걸쳐서 너무 높은 빈도수를 가지는 단어 피처를 제외
        - 정수 값 n을 지정하면 n개 이하로 나타나는 단어 피처를 제외
        - 실수 값(0.0~1.0)을 지정하면 지정한 백분율 이하 빈도수까지의 단어만 피처로 추출
    - min_df: 전체 문서에 걸쳐서 너무 낮은 빈도수를 가지는 단어 피처를 제외
        - 정수 값 n을 지정하면 n개 이하로 나타나는 단어는 피쳐 추출 제외
        

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',ngram_range=(1,2),min_df=0.05,max_df=0.85)
feature_vect = tfidf_vect.fit_transform(document_df['opinion'])

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


## 군집 수 5개로 군집

In [9]:
from sklearn.cluster import KMeans

km_cluster = KMeans(n_clusters=5,max_iter=10000,random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [10]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,2
1,bathroom_bestwestern_hotel_sfo,...,0
2,battery-life_amazon_kindle,...,1
3,battery-life_ipod_nano_8gb,...,1
4,battery-life_netbook_1005ha,...,1


- 클러스터 레이블 0인 데이터 셋 확인
- 클러스터 0은 호텔 리뷰

In [11]:
document_df[document_df['cluster_label']==0].sort_values('filename')

,filename,opinion,cluster_label
1,bathroom_bestwestern_hotel_sfo,...,0
32,room_holiday_inn_london,...,0
30,rooms_bestwestern_hotel_sfo,...,0
31,rooms_swissotel_chicago,...,0


- 클러스터 레이블 1인 데이터 셋 확인
- 클러스터 1은 포터블 전자기기 리뷰

In [12]:
document_df[document_df['cluster_label']==1].sort_values('filename')

,filename,opinion,cluster_label
2,battery-life_amazon_kindle,...,1
3,battery-life_ipod_nano_8gb,...,1
4,battery-life_netbook_1005ha,...,1
19,keyboard_netbook_1005ha,...,1
26,performance_netbook_1005ha,...,1
41,size_asus_netbook_1005ha,...,1
42,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,1
44,speed_windows7,...,1


- 클러스터 레이블 2인 데이터 셋 확인
- 클러스터 2은 gps 리뷰

In [13]:
document_df[document_df['cluster_label']==2].sort_values('filename')

,filename,opinion,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,2
5,buttons_amazon_kindle,...,2
8,directions_garmin_nuvi_255W_gps,...,2
9,display_garmin_nuvi_255W_gps,...,2
10,eyesight-issues_amazon_kindle,...,2
11,features_windows7,...,2
12,fonts_amazon_kindle,...,2
23,navigation_amazon_kindle,...,2
33,satellite_garmin_nuvi_255W_gps,...,2
34,screen_garmin_nuvi_255W_gps,...,2


- 클러스터 레이블 3인 데이터 셋 확인
- 클러스터 3은 호텔 리뷰

In [14]:
document_df[document_df['cluster_label']==3].sort_values('filename')

,filename,opinion,cluster_label
13,food_holiday_inn_london,...,3
14,food_swissotel_chicago,...,3
15,free_bestwestern_hotel_sfo,...,3
20,location_bestwestern_hotel_sfo,...,3
21,location_holiday_inn_london,...,3
24,parking_bestwestern_hotel_sfo,...,3
27,price_amazon_kindle,...,3
28,price_holiday_inn_london,...,3
38,service_bestwestern_hotel_sfo,...,3
39,service_holiday_inn_london,...,3


- 클러스터 레이블 4인 데이터 셋 확인
- 클러스터 4은 차량 리뷰

In [15]:
document_df[document_df['cluster_label']==4].sort_values('filename')

,filename,opinion,cluster_label
6,comfort_honda_accord_2008,...,4
7,comfort_toyota_camry_2007,...,4
16,gas_mileage_toyota_camry_2007,...,4
17,interior_honda_accord_2008,...,4
18,interior_toyota_camry_2007,...,4
22,mileage_honda_accord_2008,...,4
25,performance_honda_accord_2008,...,4
29,quality_toyota_camry_2007,...,4
37,seats_honda_accord_2008,...,4
47,transmission_toyota_camry_2007,...,4


## 군집수 3개로 군집

In [22]:
from sklearn.cluster import KMeans

km_cluster = KMeans(n_clusters=3,max_iter=10000,random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [23]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,0
1,bathroom_bestwestern_hotel_sfo,...,2
2,battery-life_amazon_kindle,...,0
3,battery-life_ipod_nano_8gb,...,0
4,battery-life_netbook_1005ha,...,0


In [24]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,0
1,bathroom_bestwestern_hotel_sfo,...,2
2,battery-life_amazon_kindle,...,0
3,battery-life_ipod_nano_8gb,...,0
4,battery-life_netbook_1005ha,...,0


In [35]:
document_df[document_df['cluster_label']==1].sort_values('filename')

,filename,opinion,cluster_label
6,comfort_honda_accord_2008,...,1
7,comfort_toyota_camry_2007,...,1
16,gas_mileage_toyota_camry_2007,...,1
17,interior_honda_accord_2008,...,1
18,interior_toyota_camry_2007,...,1
22,mileage_honda_accord_2008,...,1
25,performance_honda_accord_2008,...,1
29,quality_toyota_camry_2007,...,1
37,seats_honda_accord_2008,...,1
47,transmission_toyota_camry_2007,...,1


In [27]:
document_df[document_df['cluster_label']==0].sort_values('filename')

,filename,opinion,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,0
2,battery-life_amazon_kindle,...,0
3,battery-life_ipod_nano_8gb,...,0
4,battery-life_netbook_1005ha,...,0
5,buttons_amazon_kindle,...,0
8,directions_garmin_nuvi_255W_gps,...,0
9,display_garmin_nuvi_255W_gps,...,0
10,eyesight-issues_amazon_kindle,...,0
11,features_windows7,...,0
12,fonts_amazon_kindle,...,0


# 군집별 핵심 단어 추출하기

- cluster_centers_
    - 단어 피처가 군집의 중심으로부터 얼마나 가깝게 있는지를 군집 내의 피처들과의 상대 위치로 나타낸 값
    - 0~1 사이의 값으로 1에 가까울 수록 중심과 가까운 것을 의미

In [29]:
cluster_centers = km_cluster.cluster_centers_
print(cluster_centers.shape) # 3 -> 군집의 수, 4611 -> 단어 피처의 수
print(cluster_centers)

(3, 4611)
[[0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]]


[매개변수 목록]
- cluster_model : KMeans 모델 객체
- cluster_data : 리뷰 문서 데이터 및 군집 결과 값을 가지고 있는 dataframe 객체
- feature_names : 피처 카운터 벡터화 된 각 피처(단어)의 이름
- cluster_num: 군집 수
- top_n_features: 군집별 확인하고자 하는 상위 n개의 피처수

In [39]:
def get_cluster_details(cluster_model, cluster_data, feature_names, cluster_nums,top_n_features = 10):
    cluster_details = {}

    # custer_centers 배열의 값이 큰 순으로 정렬된 index 값
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    for cluster_num in range(cluster_nums):
        # {0:{} 1:{} 2:{}}의 형식
        cluster_details[cluster_num] = {}
        # {0:{'cluster':0}}
        cluster_details[cluster_num]['cluster'] = cluster_num

        top_feature_indexes = centroid_feature_ordered_ind[cluster_num,:top_n_features]

        top_features = [feature_names[ind] for ind in top_feature_indexes]

        # {0:{'cluster':0,'top_features':[....]}}
        cluster_details[cluster_num]['top_features'] = top_features
        #시리즈 객체
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        # 시리즈 객체를 리스트로 변환 .values(ndarray).tolist(python list)
        cluster_details[cluster_num]['filenames'] = filenames.values.tolist()

    return cluster_details

In [40]:
# TfidVectorizer를 통해 피처 벡터화된 피처 이름 목록 반환
feature_names = tfidf_vect.get_feature_names_out()
cluster_details = get_cluster_details(km_cluster,document_df,feature_names,3,10)

for cluster_num, cluster_detail in cluster_details.items():
    print(f'===== Cluster {cluster_num} =====')
    print('Top Features:', cluster_detail['top_features'])
    print('Review 파일명:',cluster_detail['filenames'][:5])
    print('='*40)

===== Cluster 0 =====
Top Features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Review 파일명: ['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_kindle']
===== Cluster 1 =====
Top Features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Review 파일명: ['comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_honda_accord_2008', 'interior_toyota_camry_2007']
===== Cluster 2 =====
Top Features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Review 파일명: ['bathroom_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'food_swissotel_chicago', 'free_bestwestern_hotel_sfo', 'location_bestwestern_hotel_sfo']
